## **The Battle of the Neighborhoods - Week 2**

### **Explore New York city geographical coordinates dataset**

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Link to the dataset: 
- https://geo.nyu.edu/catalog/nyu_2451_34572

First, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


The "*json*" file is downloaded and it is placed on the server. So run a "*wget*" command and access the data.

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


**Load and explore the data**

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

All the relevant data is in the features key, which is basically a list of the neighborhoods. So, define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

Take a look at the first item in this list

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

**Transform the data into a Pandas Dataframe**

The next task is essentially transforming this data of nested Python dictionaries into a Pandas Dataframe. Start by creating an empty Dataframe.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then loop through the data and fill the dataframe one row at a time.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [10]:
neighborhoods.to_csv('BON1_NYC_GEO.csv',index=False)

**Use geopy library to get the latitude and longitude values of New York City.**

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Create a map of New York with neighborhoods superimposed on top.

'*Folium*' is a great visualization library. We can zoom into the below map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

In [12]:
# create map of NewYork using latitude and longitude values
map_NewYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NewYork)  
    
map_NewYork

### **Web scrapping of Population and Demographics data of New York city from Wikipedia**

#### **A : POPULATION DATA**

Web scrapping of Population data from wikipedia page 
https://en.wikipedia.org/wiki/New_York_City

Download all the dependencies that is needed.

In [13]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

Packages installed.


In [14]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# conda install -c anaconda beautiful-soup --yes
from bs4 import BeautifulSoup # package for parsing HTML and XML documents

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Web scrapping of Population Data from wikipedia page using BeautifulSoup.

In [16]:
URL = 'https://en.wikipedia.org/wiki/Demographics_of_New_York_City'
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [17]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(r.content, 'html5lib')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('BON2_POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)


**Load data from CSV**

In [18]:
Pop_data=pd.read_csv('BON2_POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[3,7,8,9,10,11,12,13,14]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


Remove white spaces and rename columns

In [19]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte\n,Jurisdiction\n,Population\n,Landarea\n,Density\n,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,214","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN


In [20]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Landarea\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,214","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN


Replace newline('\n') from each string from left and right sides and delete unnecessary columns 

In [21]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2017']=Pop_data['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_miles']=Pop_data['square_miles'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_km']=Pop_data['square_km'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)

Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,214","122,284",412
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN


In [22]:
Pop_data.iloc[5] = Pop_data.iloc[5].shift(1,axis=0)
Pop_data.iloc[6] = Pop_data.iloc[6].shift(1,axis=0)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,NaN,City of New York,"8,336,817",842.343,302.64,783.83
6,NaN,State of New York,"19,453,561","1,731.910","47,214","122,284"
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN


Now let's remove NaN value 

In [23]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,214","122,284"
7,Sources:[14] and see individual borough articles,,,,,


and the last unnecessary row 

In [24]:
i = Pop_data[((Pop_data.Borough == 'Sources:[14] and see individual borough articles'))].index
Pop_data.drop(i)

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,214","122,284"


Save Dataframe as CSV 

In [25]:
Pop_data.to_csv('BON2_POPULATION.csv',index=False)

### **B : DEMOGRAPHICS DATA**

We will web scrap Demographics data from wikipedia page 
- https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City

Web scrapping of Demographics data from wikipedia page using BeautifulSoup.

In [26]:
URL = 'https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City'
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [27]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City').text
soup = BeautifulSoup(website_url,'html5lib')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('NYC_DEMO.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

Load data from CSV

In [28]:
Demo_data=pd.read_csv('NYC_DEMO.csv')
print('Data downloaded!')

Data downloaded!


In [29]:
Demo_data

,Year,Population,White(includes White Hispanics),%W,Non-Hispanic Whites,%ANG,Black,%B,Asian,%A,Other orMixed,%O/M,Hispanic/Latino,%H/L,Foreignborn,%FB\n
0,1900,"3,437,202","3,369,898",98.04,NaN,NaN,"60,666",1.76,"6,607",0.19,31,0.00,NaN,NaN,"1,270,080",36.95
1,1910,"4,766,883","4,669,162",97.95,NaN,NaN,"91,709",1.92,"5,669",0.12,343,0.01,NaN,NaN,"1,944,357",40.79
2,1920,"5,620,048","5,459,463",97.14,NaN,NaN,"152,467",2.71,"7,969",0.14,149,0.00,NaN,NaN,"2,028,160",36.09
3,1930,"6,930,446","6,589,377",95.08,NaN,NaN,"327,706",4.73,"12,972",0.19,391,0.01,NaN,NaN,"2,358,686",34.03
4,1940,"7,454,995","6,977,501",93.59,"6,856,586",91.97,"458,444",6.15,"17,986",0.24,"1,064",0.01,"120,915",1.62,"2,138,657",28.69
5,1950,"7,891,957","7,116,441",90.17,NaN,NaN,"747,608",9.47,"21,441",0.27,"6,467",0.08,NaN,NaN,"1,784,206",22.61
6,1960,"7,781,984","6,640,662",85.33,NaN,NaN,"1,087,931",13.98,"43,103",0.55,"10,288",0.13,NaN,NaN,"1,558,690",20.03
7,1970,"7,894,862","6,048,841",76.62,"4,969,749",62.95,"1,668,115",21.13,"94,499",1.20,"83,407",1.06,"1,278,630",16.20,"1,437,058",18.20
8,1980,"7,071,639","4,294,075",60.72,"3,668,945",51.88,"1,784,337",25.23,"231,501",3.27,"761,762",10.77,"1,406,024",19.88,"1,670,199",23.62
9,1990,"7,322,564","3,827,088",52.26,"3,163,125",43.20,"2,102,512",28.71,"512,719",7.00,"880,245",12.02,"1,783,511",24.36,"2,082,931",28.45


In [30]:
Demo_data.columns

Index(['Year', 'Population', 'White(includes White Hispanics)', '%W',
       'Non-Hispanic Whites', '%ANG', 'Black', '%B', 'Asian', '%A',
       'Other orMixed', '%O/M', 'Hispanic/Latino', '%H/L', 'Foreignborn',
       '%FB\n'],
      dtype='object')

Remove NaN values 

In [31]:
Demo_data.columns = Demo_data.columns.str.replace(' ', '')

In [32]:
Demo_data= Demo_data.fillna('')
Demo_data

,Year,Population,White(includesWhiteHispanics),%W,Non-HispanicWhites,%ANG,Black,%B,Asian,%A,OtherorMixed,%O/M,Hispanic/Latino,%H/L,Foreignborn,%FB\n
0,1900,"3,437,202","3,369,898",98.04,,,"60,666",1.76,"6,607",0.19,31,0.00,,,"1,270,080",36.95
1,1910,"4,766,883","4,669,162",97.95,,,"91,709",1.92,"5,669",0.12,343,0.01,,,"1,944,357",40.79
2,1920,"5,620,048","5,459,463",97.14,,,"152,467",2.71,"7,969",0.14,149,0.00,,,"2,028,160",36.09
3,1930,"6,930,446","6,589,377",95.08,,,"327,706",4.73,"12,972",0.19,391,0.01,,,"2,358,686",34.03
4,1940,"7,454,995","6,977,501",93.59,"6,856,586",91.97,"458,444",6.15,"17,986",0.24,"1,064",0.01,"120,915",1.62,"2,138,657",28.69
5,1950,"7,891,957","7,116,441",90.17,,,"747,608",9.47,"21,441",0.27,"6,467",0.08,,,"1,784,206",22.61
6,1960,"7,781,984","6,640,662",85.33,,,"1,087,931",13.98,"43,103",0.55,"10,288",0.13,,,"1,558,690",20.03
7,1970,"7,894,862","6,048,841",76.62,"4,969,749",62.95,"1,668,115",21.13,"94,499",1.20,"83,407",1.06,"1,278,630",16.2,"1,437,058",18.20
8,1980,"7,071,639","4,294,075",60.72,"3,668,945",51.88,"1,784,337",25.23,"231,501",3.27,"761,762",10.77,"1,406,024",19.88,"1,670,199",23.62
9,1990,"7,322,564","3,827,088",52.26,"3,163,125",43.2,"2,102,512",28.71,"512,719",7.00,"880,245",12.02,"1,783,511",24.36,"2,082,931",28.45


Save data as BON2_DEMOGRAPHICS.csv

In [33]:
Demo_data.to_csv('BON2_DEMOGRAPHICS.csv',index=False)

## **Download and Explore New York city its Restaurants & Cuisine dataset**

This data is extracted from 
- https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/data

let's first read our csv file .

In [34]:
#use necessery data
use_cols = ["DBA", "BORO", "STREET", "CUISINE DESCRIPTION", "Latitude","Longitude"]
ignore_cols = ["CAMIS", "BUILDING ","ZIPCODE","PHONE","INSPECTION DATE","ACTION","VIOLATION CODE","VIOLATION DESCRIPTION","CRITICAL FLAG","SCORE","GRADE","GRADE DATE","RECORD DATE",
               "INSPECTION TYPE","Community Board","Council District","Census Tract","BIN"]
#read csv                                                                                                  
NYC_Rest = pd.read_csv("New_York_City_Restaurant.csv", usecols=use_cols)

In [35]:
NYC_Rest.rename(columns={'DBA':'Restaurant','BORO':'Borough','STREET':'Street','CUISINE DESCRIPTION':'Cuisine'}, inplace=True)

NYC_Rest.head()

,Restaurant,Borough,Street,Cuisine,Latitude,Longitude
0,DB BISTRO MODERNE,Manhattan,WEST 44 STREET,French,40.755509,-73.981660
1,TAMAQUA,Brooklyn,EBONY COURT,"Bottled beverages, including water, sodas, jui...",40.594272,-73.930063
2,ROCKEFELLER UNIVERSITY - EMPLOYEE DINING,Manhattan,YORK AVENUE,American,40.763750,-73.956002
3,SIDE BAR,Manhattan,EAST 15 STREET,American,40.734543,-73.988053
4,LA CASA DE LOS ANOJITOS,Queens,37 AVENUE,"Latin (Cuban, Dominican, Puerto Rican, South &...",40.749656,-73.885045


In [36]:
NYC_Rest.shape

(388155, 6)

In [37]:
NYC_Rest.to_csv('NYC_Rest.csv',index=False)

In [38]:
NYCR_Data = pd.read_csv("NYC_Rest.csv")
NYCR_Data.head()

,Restaurant,Borough,Street,Cuisine,Latitude,Longitude
0,DB BISTRO MODERNE,Manhattan,WEST 44 STREET,French,40.755509,-73.981660
1,TAMAQUA,Brooklyn,EBONY COURT,"Bottled beverages, including water, sodas, jui...",40.594272,-73.930063
2,ROCKEFELLER UNIVERSITY - EMPLOYEE DINING,Manhattan,YORK AVENUE,American,40.763750,-73.956002
3,SIDE BAR,Manhattan,EAST 15 STREET,American,40.734543,-73.988053
4,LA CASA DE LOS ANOJITOS,Queens,37 AVENUE,"Latin (Cuban, Dominican, Puerto Rican, South &...",40.749656,-73.885045


Now let's discover the Borough and cuisine

In [39]:
print(NYCR_Data.Borough.unique())

['Manhattan' 'Brooklyn' 'Queens' 'Bronx' 'Staten Island' '0']


In [40]:
print(NYCR_Data.Cuisine.unique())

['French' 'Bottled beverages, including water, sodas, juices, etc.'
 'American'
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)'
 'Bakery' 'African' 'Indian' 'Spanish' 'Ethiopian' 'Italian'
 'Delicatessen' 'Irish' 'Chinese' 'Ice Cream, Gelato, Yogurt, Ices'
 'Sandwiches/Salads/Mixed Buffet' 'Café/Coffee/Tea' 'Pizza/Italian'
 'Chicken' 'Sandwiches' 'Caribbean' 'Brazilian' 'Mediterranean'
 'Pancakes/Waffles' 'Other' 'Mexican' 'Middle Eastern' 'Pizza' 'Korean'
 'Asian' 'Jewish/Kosher' 'Japanese' 'Bagels/Pretzels' 'Seafood' 'Polish'
 'Scandinavian' 'Bangladeshi' 'Juice, Smoothies, Fruit Salads' 'Greek'
 'Donuts' 'Thai' 'Tex-Mex' 'German' 'Eastern European' 'Hamburgers'
 'Australian' 'Pakistani' 'Turkish' 'Hotdogs' 'Hawaiian' 'Peruvian'
 'Russian' 'Vegetarian' 'Egyptian' 'Steak' 'Salads' 'Chinese/Japanese'
 'Vietnamese/Cambodian/Malaysia' 'Continental' 'English' 'Barbecue'
 'Hotdogs/Pretzels' 'Tapas' 'Creole' 'Creole/Cajun' 'Soul Food'
 'Fruits/Vegetables' 'Soups & Sandwi

let's count number of Borough who have restaurant.

In [41]:
NYCR_Data['Borough'].value_counts().to_frame()

,Borough
Manhattan,156676
Brooklyn,101077
Queens,81220
Bronx,35870
Staten Island,13232
0,80


The top cuisine in New York City.

In [42]:
NYCR_Data['Cuisine'].value_counts().to_frame()

,Cuisine
American,81398
Chinese,39698
Café/Coffee/Tea,19369
"Latin (Cuban, Dominican, Puerto Rican, South & Central American)",16850
Pizza,16845
Mexican,16266
Italian,15865
Japanese,14141
Caribbean,14121
Bakery,12180


## **Segmenting and Clustering Neighborhoods - Brooklyn and Manhattan**

Explore Dataset

In [43]:
NYC_Geo=pd.read_csv('BON1_NYC_GEO.csv')
print('Data downloaded!')

Data downloaded!


In [44]:
NYC_Geo.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [45]:
NYC_Geo['Borough'].value_counts().to_frame()

,Borough
Queens,81
Brooklyn,70
Staten Island,63
Bronx,52
Manhattan,40


In [46]:
NYC_Geo.shape

(306, 4)

Neighborhood has a total of 5 boroughs and 306 neighborhoods

In [47]:
print(NYC_Geo.Borough.unique())

['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


Segmenting and Clustering Neighborhoods - Brooklyn and Manhattan

In [48]:
BM_Geo = NYC_Geo.loc[(NYC_Geo['Borough'] == 'Brooklyn')|(NYC_Geo['Borough'] == 'Manhattan')]
BM_Geo = BM_Geo.reset_index(drop=True)
BM_Geo.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Brooklyn,Bay Ridge,40.625801,-74.030621
2,Brooklyn,Bensonhurst,40.611009,-73.995180
3,Brooklyn,Sunset Park,40.645103,-74.010316
4,Brooklyn,Greenpoint,40.730201,-73.954241


In [49]:
BM_Geo.shape

(110, 4)

Let's use geopy library to get the latitude and longitude values of New York City. The geograpical coordinate of New York City are 40.7308619, -73.9871558..

In [50]:
import time
start_time = time.time()

address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

print("--- %s seconds ---" % round((time.time() - start_time), 2))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.
--- 0.39 seconds ---


Create a map of Brooklyn and Manhattan with neighborhoods superimposed on top.

In [51]:
# create map of Toronto using latitude and longitude values
map_BM = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(BM_Geo['Latitude'], BM_Geo['Longitude'], BM_Geo['Borough'], BM_Geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_BM)  
    
map_BM

Extract Venues data for each neighborhoods in Brooklyn

let's use the Foursquare API to explore neighborhoods in Brooklyn and Manhattan.

In [52]:
CLIENT_ID = 'OLN1BAQQBHO234LKFIU1ZNGV4Z3O3P1GS5KIMTNPJHLX1MKL' # your Foursquare ID
CLIENT_SECRET = 'VDM5CGGVSUOGKMY21ETO4J1UAJH5QJEALQCJAIWUF2DJXR2T' # your Foursquare Secret
VERSION = '20181218' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OLN1BAQQBHO234LKFIU1ZNGV4Z3O3P1GS5KIMTNPJHLX1MKL
CLIENT_SECRET:VDM5CGGVSUOGKMY21ETO4J1UAJH5QJEALQCJAIWUF2DJXR2T


In [53]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=200, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new Dataframe

In [55]:
BM_venues = getNearbyVenues(names=BM_Geo['Neighborhood'],
                                  latitudes=BM_Geo['Latitude'],
                                  longitudes=BM_Geo['Longitude'],
                                  LIMIT=200)
BM_venues.to_csv('BM_venues.csv', sep=',', encoding='UTF8')
BM_venues.head()

Marble Hill
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [56]:
colnames = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
BM_venues = pd.read_csv('BM_venues.csv', skiprows=1, names=colnames)
BM_venues.columns = BM_venues.columns.str.replace(' ', '')
BM_venues.head()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [57]:
BM_venues.shape

(9656, 7)

Let's visualize the venues

In [58]:
def Venues_Map(Borough_name, Borough_neighborhoods):
    
    # Use geopy library to get the latitude and longitude values 
    geolocator = Nominatim(user_agent="Jupyter")
    Borough_location = geolocator.geocode(Borough_name) #'Brooklyn, NY'
    Borough_latitude = Borough_location.latitude
    Borough_longitude = Borough_location.longitude
    print('The geographical coordinates of "{}" are {}, {}.'.format(Borough_name, Borough_latitude, Borough_longitude))
    
    # To verify the number of Boroughs and Neighborhoods in the extracted data
    print('The "{}" dataframe has {} different venue types and {} neighborhoods.'.format(
          Borough_name,
          len(Borough_neighborhoods['VenueCategory'].unique()),
          len(Borough_neighborhoods['Neighborhood'].unique())))
    
    # create map of city using latitude and longitude values
    map_Borough = folium.Map(location=[Borough_latitude, Borough_longitude], zoom_start=10)

    # add markers to map
    for lat, lng, venue, category in zip(Borough_neighborhoods['VenueLatitude'], Borough_neighborhoods['VenueLongitude'], Borough_neighborhoods['Venue'], Borough_neighborhoods['VenueCategory']):
        label = '{}, {}'.format(category, venue)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=0.1,
            popup=label,
            color='red',
            fill=True,
            fill_color='#FF0000',
            fill_opacity=0.3).add_to(map_Borough)  

    return map_Borough

In [60]:
BM_venues.groupby('VenueCategory')['Venue'].count().sort_values(ascending=False)

VenueCategory
Pizza Place                                 411
Coffee Shop                                 364
Bakery                                      240
Park                                        235
Italian Restaurant                          229
Bar                                         216
Café                                        215
Grocery Store                               195
American Restaurant                         174
Mexican Restaurant                          157
Chinese Restaurant                          153
Sandwich Place                              151
Gym                                         149
Ice Cream Shop                              141
Deli / Bodega                               139
Cocktail Bar                                137
Caribbean Restaurant                        136
Gym / Fitness Center                        128
Wine Shop                                   126
Bagel Shop                                  117
Donut Shop                

Let's see how many venues were returned for each neighborhood 

In [61]:
BM_venues.groupby('Neighborhood').count()

,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Bath Beach,94,94,94,94,94,94
Battery Park City,100,100,100,100,100,100
Bay Ridge,100,100,100,100,100,100
Bedford Stuyvesant,100,100,100,100,100,100
Bensonhurst,100,100,100,100,100,100
Bergen Beach,10,10,10,10,10,10
Boerum Hill,100,100,100,100,100,100
Borough Park,55,55,55,55,55,55
Brighton Beach,92,92,92,92,92,92


In [62]:
print('There are {} uniques categories.'.format(len(BM_venues['VenueCategory'].unique())))

There are 407 uniques categories.


let's analyze Each Neighborhood

In [63]:
# one hot encoding
BM_onehot = pd.get_dummies(BM_venues[['VenueCategory']], prefix="", prefix_sep="")

#column lists before adding neighborhood
column_names = ['Neighborhood'] + list(BM_onehot.columns)

# add neighborhood column back to dataframe
BM_onehot['Neighborhood'] = BM_venues['Neighborhood'] 

# move neighborhood column to the first column
BM_onehot = BM_onehot[column_names]

BM_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant

Filtering Data 

In [64]:
restaurant_List = []
search = 'Restaurant'
for i in BM_onehot.columns :
    if search in i:
        restaurant_List.append(i)

In [65]:
col_name = []
col_name = ['Neighborhood'] + restaurant_List
BM_restaurant = BM_onehot[col_name]
BM_restaurant = BM_restaurant.iloc[:,1::]

In [66]:
BM_restaurant_grouped = BM_restaurant.groupby('Neighborhood').sum().reset_index()

In [67]:
BM_restaurant_grouped['Total'] = BM_restaurant_grouped .sum(axis=1)

In [68]:
restaurant_List

['African Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Austrian Restaurant',
 'Burmese Restaurant',
 'Cajun / Creole Restaurant',
 'Cambodian Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Chinese Restaurant',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Cuban Restaurant',
 'Czech Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Eastern European Restaurant',
 'Empanada Restaurant',
 'English Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Hawaiian Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Israeli Restaurant',
 'Italian Restaurant',
 'Japanese Curry Restaurant',
 'Japanese Restaurant',
 'Jewish Restaurant',
 'Korean Restaurant',
 'Kosher Restaurant',
 'Latin American Res

## Cluster Neighborhoods and Examine Clusters

let's find K-means using sklearn

In [73]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)
from sklearn.cluster import KMeans
BM_grouped_clustering = BM_restaurant_grouped.drop('Neighborhood', 1)

for n_cluster in range(2, 10):
    kmeans = KMeans(n_clusters=n_cluster).fit(BM_grouped_clustering)
    label = kmeans.labels_
    sil_coeff = metrics.silhouette_score(BM_grouped_clustering, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

For n_clusters=2, The Silhouette Coefficient is 0.3900812349669867
For n_clusters=3, The Silhouette Coefficient is 0.31826315229505725
For n_clusters=4, The Silhouette Coefficient is 0.2315935861399036
For n_clusters=5, The Silhouette Coefficient is 0.25062509293729723
For n_clusters=6, The Silhouette Coefficient is 0.18384450957034676
For n_clusters=7, The Silhouette Coefficient is 0.19325491635903347
For n_clusters=8, The Silhouette Coefficient is 0.19180860164398658
For n_clusters=9, The Silhouette Coefficient is 0.1739708120337305


Run k-means to cluster the neighborhood into high accuracy Silhouette Coefficient cluster.it's 2 clusters

In [74]:
# set number of clusters
kclusters = 2

BM_grouped_clustering = BM_restaurant_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(BM_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
      dtype=int32)

In [75]:
BM_results = pd.DataFrame(kmeans.cluster_centers_)
BM_results.columns = BM_grouped_clustering.columns
BM_results.index = ['cluster0','cluster1']
BM_results['Total Sum'] = BM_results.sum(axis = 1)
BM_results

,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Total,Total Sum
cluster0,0.106061,1.787879,1.363636e-01,0.090909,0.484848,0.136364,0.075758,1.515152e-02,4.545455e-02,4.545455e-02,0.090909,1.378788,0.045455,1.575758,1.515152e-02,1.060606e-01,0.272727,3.030303e-02,9.090909e-02,0.121212,0.181818,1.060606e-01,4.545455e-02,0.106061,0.181818,0.454545,1.212121e-01,0.606061,0.090909,0.454545,0.121212,7.575758e-02,6.060606e-02,0.742424,4.545455e-02,2.712121,3.030303e-02,1.166667,3.030303e-02,0.287879,0.045455,0.727273,1.060606e-01,6.060606e-02,0.469697,2.045455,0.424242,0.015152,9.090909e-02,0.666667,1.515152e-02,1.515152e-02,6.060606e-02,0.151515,1.515152e-02,2.424242e-01,0.803030,0.257576,3.030303e-02,0.954545,1.515152e-02,0.060606,0.090909,0.409091,0.439394,1.272727,3.030303e-02,0.060606,3.030303e-02,0.409091,1.015152,6.938894e-18,4.545455e-02,0.287879,4.545455e-02,1.515152e-02,0.681818,1.515152e-02,0.454545,26.848485,53.696970
cluster1,0.022727,1.272727,-6.938894e-17,0.022727,0.068182,0.022727,0.022727,-1.734723e-18,3.469447e-18,3.469447e-18,0.022727,1.022727,0.045455,1.113636,-1.734723e-18,1.387779e-17,0.090909,-3.469447e-18,6.938894e-18,0.022727,0.045455,1.387779e-17,3.469447e-18,0.022727,0.204545,0.954545,-1.387779e-17,0.363636,0.045455,0.045455,0.022727,2.775558e-17,-6.938894e-18,0.136364,3.469447e-18,1.136364,-3.469447e-18,0.545455,-3.469447e-18,0.045455,0.045455,0.295455,1.387779e-17,-6.938894e-18,0.295455,0.500000,0.250000,0.022727,6.938894e-18,0.181818,-1.734723e-18,-1.734723e-18,-6.938894e-18,0.159091,-1.734723e-18,-2.775558e-17,0.409091,0.090909,-3.469447e-18,0.636364,-1.734723e-18,0.022727,0.090909,0.113636,0.136364,0.522727,-3.469447e-18,0.022727,-3.469447e-18,0.068182,0.181818,2.272727e-02,3.469447e-18,0.068182,3.469447e-18,-1.734723e-18,0.113636,-1.734723e-18,0.090909,11.659091,23.318182


The Total and Total Sum of cluster0 has smallest value. It shows that the market is not saturated.

In [76]:
BM_results_merged = pd.DataFrame(BM_restaurant_grouped['Neighborhood'])

BM_results_merged['Total'] = BM_restaurant_grouped['Total']
BM_results_merged = BM_results_merged.assign(Cluster_Labels = kmeans.labels_)

In [77]:
print(BM_results_merged.shape)
BM_results_merged

(110, 3)


,Neighborhood,Total,Cluster_Labels
0,Bath Beach,30,0
1,Battery Park City,13,1
2,Bay Ridge,36,0
3,Bedford Stuyvesant,20,0
4,Bensonhurst,28,0
5,Bergen Beach,5,1
6,Boerum Hill,19,1
7,Borough Park,9,1
8,Brighton Beach,21,0
9,Broadway Junction,9,1


Merge BM_results_merged with BM_Geo

In [78]:
BM_merged = BM_Geo

BM_merged = BM_merged.join(BM_results_merged.set_index('Neighborhood'), on='Neighborhood')

print(BM_merged.shape)
BM_merged.head(10) # check the last columns!

(110, 6)


,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,14,1
1,Brooklyn,Bay Ridge,40.625801,-74.030621,36,0
2,Brooklyn,Bensonhurst,40.611009,-73.995180,28,0
3,Brooklyn,Sunset Park,40.645103,-74.010316,39,0
4,Brooklyn,Greenpoint,40.730201,-73.954241,23,0
5,Brooklyn,Gravesend,40.595260,-73.973471,9,1
6,Brooklyn,Brighton Beach,40.576825,-73.965094,21,0
7,Brooklyn,Sheepshead Bay,40.586890,-73.943186,31,0
8,Brooklyn,Manhattan Terrace,40.614433,-73.957438,17,1
9,Brooklyn,Flatbush,40.636326,-73.958401,32,0


Finally, let's visualize the resulting clusters

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(BM_merged['Latitude'], BM_merged['Longitude'], BM_merged['Neighborhood'], BM_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### List Neighborhoods of Interest in New York City

Cluster 1 : Saturated Markets

In [80]:
BM_merged[BM_merged['Cluster_Labels'] == 1].reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,14,1
1,Brooklyn,Gravesend,40.595260,-73.973471,9,1
2,Brooklyn,Manhattan Terrace,40.614433,-73.957438,17,1
3,Brooklyn,East Flatbush,40.641718,-73.936103,6,1
4,Brooklyn,Windsor Terrace,40.656946,-73.980073,18,1
5,Brooklyn,Brownsville,40.663950,-73.910235,11,1
6,Brooklyn,Brooklyn Heights,40.695864,-73.993782,18,1
7,Brooklyn,Carroll Gardens,40.680540,-73.994654,18,1
8,Brooklyn,Red Hook,40.676253,-74.012759,10,1
9,Brooklyn,Cypress Hills,40.682391,-73.876616,12,1


Cluster 0 : Untapped Markets

In [82]:
BM_merged[BM_merged['Total'] == 0].reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels


There are no Untapped Markets for Restaurant business in Brooklyn and Manhattan.

#### Segmenting and Clustering Neighborhoods - Bronx, Queens and Staten Island

In [85]:
BQS_Geo = NYC_Geo.loc[(NYC_Geo['Borough'] == 'Bronx')|(NYC_Geo['Borough'] == 'Queens')|(NYC_Geo['Borough'] == 'Staten Island')]
BQS_Geo = BQS_Geo.reset_index(drop=True)
BQS_Geo.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [86]:
BQS_Geo.shape

(196, 4)

Create a map of Bronx, Queens and Staten Island 

In [87]:
# create map of Toronto using latitude and longitude values
map_BQS = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(BQS_Geo['Latitude'], BQS_Geo['Longitude'], BQS_Geo['Borough'], BQS_Geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_BQS)  
    
map_BQS

Explore Neighborhoods in Bronx, Queens and Staten Island

In [88]:
BQS_venues = getNearbyVenues(names=BQS_Geo['Neighborhood'],
                                  latitudes=BQS_Geo['Latitude'],
                                  longitudes=BQS_Geo['Longitude'],
                                  LIMIT=200)
BQS_venues.to_csv('BQS_venues.csv', sep=',', encoding='UTF8')
BQS_venues.head()

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cam

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant


Visualize the BQS_Venues data

In [105]:
BQS_venues.groupby('Venue Category')['Venue'].count().sort_values(ascending=False)

Venue Category
Pizza Place                                 631
Donut Shop                                  343
Deli / Bodega                               325
Italian Restaurant                          298
Sandwich Place                              279
Pharmacy                                    279
Chinese Restaurant                          252
Bank                                        236
Bakery                                      228
Park                                        216
Coffee Shop                                 209
Fast Food Restaurant                        205
Grocery Store                               205
Bar                                         202
Ice Cream Shop                              183
Supermarket                                 180
Mexican Restaurant                          177
Bus Stop                                    170
Diner                                       169
Bus Station                                 137
American Restaurant      

Let's check how many venues were returned for each neighborhood

In [106]:
BQS_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,66,66,66,66,66,66
Annadale,19,19,19,19,19,19
Arden Heights,22,22,22,22,22,22
Arlington,22,22,22,22,22,22
Arrochar,23,23,23,23,23,23
Arverne,34,34,34,34,34,34
Astoria,100,100,100,100,100,100
Astoria Heights,76,76,76,76,76,76
Auburndale,100,100,100,100,100,100


In [108]:
print('There are {} uniques categories.'.format(len(BQS_venues['Venue Category'].unique())))

There are 386 uniques categories.


Analyze Each Neighborhood

In [111]:
# one hot encoding
BQS_onehot = pd.get_dummies(BQS_venues[['Venue Category']], prefix="", prefix_sep="")

#column lists before adding neighborhood
column_names = ['Neighborhood'] + list(BQS_onehot.columns)

# add neighborhood column back to dataframe
BQS_onehot['Neighborhood'] = BQS_venues['Neighborhood'] 

# move neighborhood column to the first column
BQS_onehot = BQS_onehot[column_names]

BQS_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Emergency Room,Empanada Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Outlet Mall,Paella Restaurant,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Public 

In [112]:
restaurant_List1 = []
search = 'Restaurant'
for i in BQS_onehot.columns :
    if search in i:
        restaurant_List1.append(i)

In [113]:
restaurant_List1

['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Brazilian Restaurant',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Cuban Restaurant',
 'Czech Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Eastern European Restaurant',
 'Egyptian Restaurant',
 'Empanada Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Himalayan Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Kosher Restaurant',
 'Latin American Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eas

In [114]:
col_name = []
col_name = ['Neighborhood'] + restaurant_List1
BQS_restaurant = BQS_onehot[col_name]
BQS_restaurant = BQS_restaurant.iloc[:,1::]

In [115]:
BQS_restaurant_grouped = BQS_restaurant.groupby('Neighborhood').sum().reset_index()

In [116]:
BQS_restaurant_grouped['Total'] = BQS_restaurant_grouped .sum(axis=1)

Cluster Neighborhoods and Examine Clusters

In [117]:
BQS_grouped_clustering = BQS_restaurant_grouped.drop('Neighborhood', 1)

for n_cluster in range(2, 10):
    kmeans = KMeans(n_clusters=n_cluster).fit(BQS_grouped_clustering)
    label = kmeans.labels_
    sil_coeff = metrics.silhouette_score(BQS_grouped_clustering, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

For n_clusters=2, The Silhouette Coefficient is 0.5188996477290404
For n_clusters=3, The Silhouette Coefficient is 0.37265651149588186
For n_clusters=4, The Silhouette Coefficient is 0.356033385015751
For n_clusters=5, The Silhouette Coefficient is 0.3785546868955611
For n_clusters=6, The Silhouette Coefficient is 0.33146717555328814
For n_clusters=7, The Silhouette Coefficient is 0.28576873697865707
For n_clusters=8, The Silhouette Coefficient is 0.3141941050509995
For n_clusters=9, The Silhouette Coefficient is 0.21329851374362868


Run k-means to cluster the neighborhood 

In [118]:
# set number of clusters
kclusters = 2

BQS_grouped_clustering = BQS_restaurant_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(BQS_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [119]:
BQS_results = pd.DataFrame(kmeans.cluster_centers_)
BQS_results.columns = BQS_grouped_clustering.columns
BQS_results.index = ['cluster0','cluster1']
BQS_results['Total Sum'] = BQS_results.sum(axis = 1)
BQS_results

,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern Greek Restaurant,New American Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Salvadoran Restaurant,Seafood Restaurant,Shanghai Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Total,Total Sum
cluster0,0.045455,0.045455,1.181818,0.181818,2.500000e-01,0.886364,2.272727e-02,0.181818,0.113636,1.590909e-01,0.50,2.159091,0.113636,0.045455,0.090909,2.272727e-02,0.159091,0.227273,0.181818,2.272727e-02,0.113636,0.022727,1.113636,0.204545,0.159091,0.022727,-3.469447e-18,0.795455,0.136364,6.818182e-02,1.136364e-01,1.454545,6.818182e-02,2.659091,0.795455,1.590909,4.545455e-02,1.25,1.136364e-01,0.204545,2.181818,0.386364,2.272727e-02,0.181818,0.022727,0.022727,2.272727e-02,0.477273,0.022727,-3.469447e-18,1.363636e-01,0.590909,-3.469447e-18,4.545455e-02,0.636364,2.045455e-01,0.590909,0.204545,0.818182,0.068182,0.909091,1.136364e-01,0.022727,0.113636,-6.938894e-18,1.000000,0.022727,1.590909e-01,2.272727e-01,0.204545,2.272727e-02,0.340909,27.295455,54.590909
cluster1,0.006667,0.026667,0.566667,0.020000,1.387779e-16,0.253333,1.040834e-17,0.006667,0.020000,6.245005e-17,0.56,1.046667,0.006667,0.013333,0.006667,1.040834e-17,0.013333,0.006667,0.060000,1.040834e-17,0.013333,0.006667,1.040000,0.026667,0.013333,0.033333,6.666667e-03,0.060000,0.026667,4.336809e-17,6.245005e-17,0.180000,4.336809e-17,1.206667,0.253333,0.040000,2.081668e-17,0.28,6.245005e-17,0.080000,0.540000,0.093333,1.040834e-17,0.060000,0.013333,0.006667,1.040834e-17,0.066667,0.006667,6.666667e-03,8.673617e-17,0.340000,6.666667e-03,2.081668e-17,0.186667,1.040834e-16,0.053333,0.040000,0.353333,0.053333,0.313333,6.245005e-17,0.006667,0.033333,1.333333e-02,0.133333,0.006667,6.245005e-17,1.249001e-16,0.053333,1.040834e-17,0.033333,8.300000,16.600000


The Total and Total Sum of cluster0 has smallest value. It shows that the market is not saturated.

In [122]:
BQS_results_merged = pd.DataFrame(BQS_restaurant_grouped['Neighborhood'],)

BQS_results_merged['Total'] = BQS_restaurant_grouped['Total']
BQS_results_merged = BQS_results_merged.assign(Cluster_Labels = kmeans.labels_)

In [123]:
print(BQS_results_merged.shape)
BQS_results_merged

(194, 3)


,Neighborhood,Total,Cluster_Labels
0,Allerton,11,1
1,Annadale,2,1
2,Arden Heights,3,1
3,Arlington,3,1
4,Arrochar,5,1
5,Arverne,2,1
6,Astoria,30,0
7,Astoria Heights,11,1
8,Auburndale,34,0
9,Bay Terrace,14,1


Merge BQS_results_merged with BQS_Geo

In [124]:
BQS_merged = BQS_Geo

BQS_merged = BQS_merged.join(BQS_results_merged.set_index('Neighborhood'), on='Neighborhood')

print(BQS_merged.shape)
BQS_merged.head(10) # check the last columns!

(196, 6)


,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Bronx,Wakefield,40.894705,-73.847201,8,1
1,Bronx,Co-op City,40.874294,-73.829939,13,1
2,Bronx,Eastchester,40.887556,-73.827806,12,1
3,Bronx,Fieldston,40.895437,-73.905643,6,1
4,Bronx,Riverdale,40.890834,-73.912585,9,1
5,Bronx,Kingsbridge,40.881687,-73.902818,19,0
6,Bronx,Woodlawn,40.898273,-73.867315,6,1
7,Bronx,Norwood,40.877224,-73.879391,16,1
8,Bronx,Williamsbridge,40.881039,-73.857446,15,1
9,Bronx,Baychester,40.866858,-73.835798,15,1


Finally, let's visualize the resulting clusters

In [126]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(BQS_merged['Latitude'], BQS_merged['Longitude'], BQS_merged['Neighborhood'], BQS_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### List Neighborhoods of Interest in New York City - Bronx, Queens and Staten Island

Cluster 1 : Saturated Markets

In [127]:
BQS_merged[BQS_merged['Cluster_Labels'] == 1].reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Bronx,Wakefield,40.894705,-73.847201,8,1
1,Bronx,Co-op City,40.874294,-73.829939,13,1
2,Bronx,Eastchester,40.887556,-73.827806,12,1
3,Bronx,Fieldston,40.895437,-73.905643,6,1
4,Bronx,Riverdale,40.890834,-73.912585,9,1
5,Bronx,Woodlawn,40.898273,-73.867315,6,1
6,Bronx,Norwood,40.877224,-73.879391,16,1
7,Bronx,Williamsbridge,40.881039,-73.857446,15,1
8,Bronx,Baychester,40.866858,-73.835798,15,1
9,Bronx,Pelham Parkway,40.857413,-73.854756,10,1


Cluster 0 : Untapped Markets

In [129]:
BQS_merged[BQS_merged['Total'] == 0].reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Staten Island,Todt Hill,40.597069,-74.111329,0,1
1,Staten Island,Port Ivory,40.639683,-74.174645,0,1
2,Staten Island,Butler Manor,40.506082,-74.229504,0,1
3,Staten Island,Bloomfield,40.605779,-74.187256,0,1
